In [4]:
import pandas as pd
import numpy as np
import re

## Goal is to deal with object columns

What does that mean?
* some object columns are mostly numeric but because of some string, ie "11 years", this makes the entire column object.
* random "." in the data converts the column to objects

We can either drop or fix the column 

In [81]:
df = pd.read_csv('raw_patient_data.csv', low_memory=False)

In [82]:
df.dtypes.value_counts()

int64      371
object      74
float64     72
dtype: int64

## 119 object columns, lets take a look

In [83]:
df_objects = df.select_dtypes('object')

In [84]:
df_objects.head()

,ices_caseid,ices_b8cannabis,ices_ivtimes,ices_ivshare,ices_numsex,ices_dayspartnerconflict,ices_numpaidwork,ices_ofvehicleday,ices_mealhour,ices_osathour,...,ices_whereother,ices_legalprescriptmj,ices_salivadate,ices_salivanumber,ices_doseaddcomm2,ices_urinelabel,ices_addcomm3,ices_addcomm6,ices_addcomm9,ices_addcomm12
0,021-0001,0,NaN,.,0,NaN,0,0,2,1,...,NaN,NaN,36:00.0,POST-021-0001-S,NaN,NaN,NaN,NaN,NaN,NaN
1,021-0002,30,NaN,.,0,NaN,0,0,15,24,...,NaN,NaN,27:00.0,POST-021-0002-S,NaN,NaN,NaN,NaN,NaN,NaN
2,021-0003,0,NaN,.,0,NaN,0,0,3,27,...,NaN,NaN,11:00.0,POST-021-0003-S,NaN,NaN,NaN,NaN,NaN,NaN
3,021-0004,0,NaN,.,0,NaN,20,0,0.5,0.3,...,NaN,NaN,38:00.0,POST-021-0004-S,NaN,NaN,NaN,NaN,NaN,"Transferred to another provider @ March 7, 2019"
4,021-0006,0,NaN,.,0,NaN,0,0,72,5,...,NaN,NaN,18:00.0,POST-021-0006-S,NaN,NaN,NaN,NaN,NaN,NaN


### Dealing with "." in the data

In [85]:
# iterate through the df_objects dataframe and replace any . with a 0

for col in df_objects.columns:
    df_objects.loc[:,col] = df_objects[col].replace('.',0).copy()

C:\Users\Hafez\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [86]:
df_objects.head()

,ices_caseid,ices_b8cannabis,ices_ivtimes,ices_ivshare,ices_numsex,ices_dayspartnerconflict,ices_numpaidwork,ices_ofvehicleday,ices_mealhour,ices_osathour,...,ices_whereother,ices_legalprescriptmj,ices_salivadate,ices_salivanumber,ices_doseaddcomm2,ices_urinelabel,ices_addcomm3,ices_addcomm6,ices_addcomm9,ices_addcomm12
0,021-0001,0,NaN,0,0,NaN,0,0,2,1,...,NaN,NaN,36:00.0,POST-021-0001-S,NaN,NaN,NaN,NaN,NaN,NaN
1,021-0002,30,NaN,0,0,NaN,0,0,15,24,...,NaN,NaN,27:00.0,POST-021-0002-S,NaN,NaN,NaN,NaN,NaN,NaN
2,021-0003,0,NaN,0,0,NaN,0,0,3,27,...,NaN,NaN,11:00.0,POST-021-0003-S,NaN,NaN,NaN,NaN,NaN,NaN
3,021-0004,0,NaN,0,0,NaN,20,0,0.5,0.3,...,NaN,NaN,38:00.0,POST-021-0004-S,NaN,NaN,NaN,NaN,NaN,"Transferred to another provider @ March 7, 2019"
4,021-0006,0,NaN,0,0,NaN,0,0,72,5,...,NaN,NaN,18:00.0,POST-021-0006-S,NaN,NaN,NaN,NaN,NaN,NaN


We will investigate `ices_osathour`

In [87]:
col = 'ices_osathour'

In [88]:
df_objects[col].value_counts()

0.5      448
24       307
1         93
25        62
26        61
        ... 
29.5       1
37         1
73         1
0.166      1
40         1
Name: ices_osathour, Length: 107, dtype: int64

Clearly we have a bunch of incorrect inputs, we can convert them to numeric and that will change them to nulls. We will do this because it's the easiest way to deal with them lol... let's check how many we can convert to numeric.

In [89]:
sum(pd.to_numeric(df_objects['ices_osathour'], errors='coerce').notnull())

1733

Cool, 1635 out of 1734.. not bad. We can do the same for every other column and then evaluate whether we actually want to continue in this way or just simply drop the column. Some columns have too many nulls even after converting them, so we can just get rid of them.

Let's columns that have too many nulls.. we can say any column with over 800 nulls we drop

In [90]:
for col in df_objects.columns:
    if sum(pd.to_numeric(df_objects[col], errors='coerce').isnull()) > 800:
        df_objects.drop(col,axis=1,inplace=True)

C:\Users\Hafez\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


We can check how many nulls we still have AFTER we convert to numeric.

In [91]:
num_nulls = {}
for col in df_objects.columns:
    num_nulls[col] = sum(pd.to_numeric(df_objects[col], errors='coerce').isnull())

In [92]:
sorted(num_nulls.items(), key=lambda x: x[1], reverse=True)

[('ices_dayspartnerconflict', 722),
 ('ices_regularuse', 201),
 ('ices_mealhour', 31),
 ('ices_ivshare', 2),
 ('ices_ofvehicleday', 1),
 ('ices_osathour', 1),
 ('ices_length6', 1),
 ('ices_b8cannabis', 0),
 ('ices_numsex', 0),
 ('ices_numpaidwork', 0)]

Cool, now we have a list of features that we might be able to recover from all the nulls.

Okay well now this is a bit complicated but I'll break it down real quick:

So we want to move the new numeric columns into the original dataframe and then write that new dataframe to a csv file. We have a df_objects which contains the object columns we might want to keep. We will map the df_object to df then drop the columns that remain. 

We started with 119 object columns --> we now have 32.

In [93]:
df.drop(df.loc[:,~df.columns.isin(df_objects.columns)].select_dtypes('object').columns, axis=1, inplace=True)

Let me explain what just happened:

* `df.loc[:,~df.columns.isin(df_objects.columns)]` will select columns that are not in df_objects
* `.select_dtypes('object').columns` will select object columns
* `df.drop()` will drop the result

tl;dr we dropped object columns that were not in df_objects :)

We should convert columns into numerics first then move it from df_objects to df

In [94]:
for col in df_objects.columns:
    df_objects.loc[:, col] = pd.to_numeric(df_objects[col], errors='coerce')

C:\Users\Hafez\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\Hafez\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [95]:
df_objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ices_b8cannabis           1734 non-null   int64  
 1   ices_ivshare              1732 non-null   float64
 2   ices_numsex               1734 non-null   int64  
 3   ices_dayspartnerconflict  1012 non-null   float64
 4   ices_numpaidwork          1734 non-null   float64
 5   ices_ofvehicleday         1733 non-null   float64
 6   ices_mealhour             1703 non-null   float64
 7   ices_osathour             1733 non-null   float64
 8   ices_length6              1733 non-null   float64
 9   ices_regularuse           1533 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 135.6 KB


NOW we can move the final df_object into the original df.

In [96]:
for col in df_objects.columns:
    df.loc[:, col] = df_objects.loc[:, col].copy()

In [97]:
df.dtypes.value_counts()

int64      373
float64     80
dtype: int64

### Wow we did it, we converted a whole bunch of objects to floats/ints BUTTTT we still have nulls that we need to take care of :P

In [98]:
df.to_csv('patient_data_numeric_edited.csv', index=False)

# Regular Expression Exp

In [31]:
col = 'ices_mentalhealthmeds'

df[col]

0                            No
1                          None
2              valium , effexor
3                            No
4         Citraline, Olanzapine
                 ...           
1729                         No
1730    Amitriptyline, cipralex
1731                       none
1732                          0
1733                         no
Name: ices_mentalhealthmeds, Length: 1734, dtype: object

If record has 'no' or 'none' or any variation of this, then they do not take meds, therefore 0. If they do have a drug we are hoping the drug doesn't have 'no' in it.

In [61]:
type(df['ices_mentalhealthmeds'].iloc[1732])

str

In [46]:
no = 'no'

no, noo, none, NO, no9812398123981iuehfjka

In [71]:
string = 'yes'

In [72]:
def has_no(col):
    try:
        if len(re.findall(r'^[\(no\)0]*', col.lower())) == 1:
            return 0
        else:
            return 1
    except:
        return 0
        


In [73]:
has_no(string)

0

In [57]:
df['ices_mentalhealthmeds'].apply(has_no)

0       0
1       0
2       1
3       0
4       1
       ..
1729    0
1730    1
1731    0
1732    1
1733    0
Name: ices_mentalhealthmeds, Length: 1734, dtype: int64

In [83]:
for col in df_objects:
    df.loc[:,col] = df_objects.loc[:,col]

In [51]:
no = 'yes'

In [71]:
def has_no(col):
    try:
        if len(re.findall(r'^no*',col.lower())):
            return 0
        return 1
    except:
        return 0

In [57]:
has_no(no)

1

In [72]:
df_objects.loc[:,'ices_mentalhealthmeds_bin'] = df_objects['ices_mentalhealthmeds'].apply(has_no).copy()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [76]:
df_objects.ices_mentalhealthmeds_bin.value_counts()

0    969
1    765
Name: ices_mentalhealthmeds_bin, dtype: int64

In [9]:
sorted(num_nulls.items(), key=lambda x: x[1], reverse=True)

[('ices_caseid', 1734),
 ('ices_addcomm1', 1734),
 ('ices_othereth', 1734),
 ('ices_lastworking', 1734),
 ('ices_howother', 1734),
 ('ices_datelastdose', 1734),
 ('ices_otherprescript', 1734),
 ('ices_b9name', 1734),
 ('ices_urinestartbase', 1734),
 ('ices_urineendbase', 1734),
 ('ices_dose3', 1734),
 ('ices_urinestart3', 1734),
 ('ices_urineend3', 1734),
 ('ices_dose6', 1734),
 ('ices_urinestart6', 1734),
 ('ices_urineend6', 1734),
 ('ices_dose9', 1734),
 ('ices_urinestart9', 1734),
 ('ices_urineend9', 1734),
 ('ices_dose12', 1734),
 ('ices_urinestart12', 1734),
 ('ices_urineend12', 1734),
 ('ices_postgenoa', 1734),
 ('ices_ethmixed', 1734),
 ('ices_occupation', 1734),
 ('ices_nameofscript', 1734),
 ('ices_nocontinuescript', 1734),
 ('ices_opioiddependence', 1734),
 ('ices_howotherdepend', 1734),
 ('Ices_currentopoth', 1734),
 ('ices_edreasonother', 1734),
 ('ices_obtainnal', 1734),
 ('ices_subvape', 1734),
 ('ices_otherosat', 1734),
 ('ices_goalsosat', 1734),
 ('ices_anyotherosat', 1

1) try pd.to_numerics(error='coerce') --> converts any non-numeric entry to a NULL
2) count the nulls
3) if the nulls are really small then dont drop the column



* ices_daysfriendsconflict : convert to 0
* ices_regularuse : convert to 0
* ices_totalyearsused : split string, if num take num else 0


In [10]:
col = 'ices_weight'

In [14]:
sum(pd.to_numeric(df_objects[col], errors='coerce').isna())

14

In [20]:
df.loc[pd.to_numeric(df[col], errors='coerce').isnull(),col] = 0

In [21]:
df.loc[pd.to_numeric(df[col], errors='coerce').isnull(),col]

Series([], Name: ices_weight, dtype: object)

In [16]:
df[col]

0        68
1        84
2        52
3        66
4        61
       ... 
1729     99
1730     66
1731     66
1732    NaN
1733     54
Name: ices_weight, Length: 1734, dtype: object

In [17]:
pd.to_numeric(df[col], errors='coerce').isnull()


0       False
1       False
2       False
3       False
4       False
        ...  
1729    False
1730    False
1731    False
1732     True
1733    False
Name: ices_weight, Length: 1734, dtype: bool

In [ ]:
def print_nonnumeric(col):
    

In [85]:
df.to_csv('patient_data_trimmed_numericed.csv')

In [87]:
df_obj

NameError: name 'df_obj' is not defined